In [7]:
import pandas as pd # type: ignore
import yfinance as yf # type: ignore
from yahoofinancials import YahooFinancials # type: ignore
import numpy as np # type: ignore
import os

os.chdir('/Users/fulinq/Documents/KMITL/FinancialEngineering/PortfolioSelection')

pd.set_option('display.max_columns', 74)
pd.set_option('display.width', 500)

In [3]:
#get uppered stock name for yfinance

# lst = list()
# while True:
#     inp = input().strip()
#     if inp == '':
#         break
#     inp = inp.upper()           # US Stock
#     # inp = inp.upper() + '.BK'   # Thai Stock
#     lst.append(inp)

# print(lst)

In [8]:
lst = ['AAPL', 'AMT', 'AMZN', 'APD', 'BAC', 'BX', 'CCZ', 'EP', 'JNJ', 'JPM', 'PCG', 'SHEL', 'T', 'TMO', 'TSLA', 'TSM', 'VRTX', 'VZ', 'WM', 'XOM']
print(lst)

['AAPL', 'AMT', 'AMZN', 'APD', 'BAC', 'BX', 'CCZ', 'EP', 'JNJ', 'JPM', 'PCG', 'SHEL', 'T', 'TMO', 'TSLA', 'TSM', 'VRTX', 'VZ', 'WM', 'XOM']


In [9]:
stockList = ['AAPL', 'AMT', 'AMZN', 
             'APD', 'BAC', 'BX', 
             'CCZ', 'EP', 'JNJ', 
             'JPM', 'PCG', 'SHEL', 
             'T', 'TMO', 'TSLA', 
             'TSM', 'VRTX', 'VZ', 
             'WM', 'XOM']
index = '^IXIC'
stockList.insert(0, index)
fileName = 'RawPrice.xlsx'
sDate = '2021-01-01'  #yyyy-mm-dd
eDate = '2025-01-31'  #yyyy-mm-dd

In [ ]:
df = pd.DataFrame()

for i, inp in enumerate(stockList):
    selected_df = yf.download(inp, 
                              start=sDate, 
                              end=eDate, 
                              interval='1mo', 
                              rounding=2, 
                              progress=False)
    
    # Flat the multiindex
    if isinstance(selected_df.columns, pd.MultiIndex):
        selected_df.columns = selected_df.columns.get_level_values(0)
    
    # Remove ticker
    if 'Price' in selected_df.columns:
        selected_df = selected_df[selected_df['Price'] != 'Ticker']
    
    # Continuously Compound Rate
    selected_df['Change'] = np.log(selected_df['Close'] / selected_df['Close'].shift(1))
    
    # Change date to column by resetting te index
    selected_df.reset_index(inplace=True)
    
    # initial date in df for merging
    if i == 0:
        df['Date'] = selected_df['Date']
    
    mer_df = selected_df[['Date', 
                          'Close', 
                          'Change'
                          ]].copy()
    mer_df = mer_df.rename(columns={'Close': 'Close ' + inp, 
                                    'Change': inp})
    
    df = pd.merge(df, mer_df, on='Date', how='outer')

# First row's change is NaN
df = df.iloc[1:]

print(df.head())

# Save to Excel
with pd.ExcelWriter(fileName, engine='openpyxl', mode='w') as writer:
    df.to_excel(writer, sheet_name='Combined', index=False)

        Date  Close ^IXIC     ^IXIC  Close AAPL      AAPL  Close AMT       AMT  Close AMZN      AMZN  Close APD       APD  Close BAC       BAC  Close BX        BX  Close CCZ       CCZ  Close EP        EP  Close JNJ       JNJ  Close JPM       JPM  Close PCG       PCG  Close SHEL      SHEL  Close T         T  Close TMO       TMO  Close TSLA      TSLA  Close TSM       TSM  Close VRTX      VRTX  Close VZ        VZ  Close WM        WM  Close XOM       XOM
1 2021-02-01     13192.35  0.009265      118.57 -0.084619     193.17 -0.050669      154.65 -0.035945     231.84 -0.042684      31.47  0.157280     60.05  0.029919      56.07 -0.100695      2.00  0.145026     141.69 -0.029006     132.26  0.141491      10.47 -0.084222       35.46  0.106426    15.89 -0.003142     445.78 -0.124396      225.17 -0.161023     117.37  0.035734      212.55 -0.074894     43.50  0.020672    104.02 -0.003934      46.07  0.192839
2 2021-03-01     13246.87  0.004124      119.62  0.008817     213.67  0.100862      154.70

In [10]:
df = pd.DataFrame()

for i, inp in enumerate(stockList):
    selected_df = yf.download(inp, 
                              start=sDate, 
                              end=eDate, 
                              interval='1mo', 
                              rounding=2, 
                              progress=False)
    
    # Flat the multiindex
    if isinstance(selected_df.columns, pd.MultiIndex):
        selected_df.columns = selected_df.columns.get_level_values(0)
    
    # Remove ticker
    if 'Price' in selected_df.columns:
        selected_df = selected_df[selected_df['Price'] != 'Ticker']
    
    # Continuously Compound Rate
    selected_df['Change'] = np.log(selected_df['Close'] / selected_df['Close'].shift(1))
    
    # Change date to column by resetting te index
    selected_df.reset_index(inplace=True)
    
    # initial date in df for merging
    if i == 0:
        df['Date'] = selected_df['Date']
    
    mer_df = selected_df[['Date', 
                          'Close', 
                          ]].copy()
    mer_df = mer_df.rename(columns={'Close': inp})
    
    df = pd.merge(df, mer_df, on='Date', how='outer')

# First row's change is NaN
df = df.iloc[1:]

print(df.head())

# Save to Excel
with pd.ExcelWriter(fileName, engine='openpyxl', mode='w') as writer:
    df.to_excel(writer, sheet_name='Combined', index=False)

        Date     ^IXIC    AAPL     AMT    AMZN     APD    BAC     BX    CCZ    EP     JNJ     JPM    PCG   SHEL      T     TMO    TSLA     TSM    VRTX     VZ      WM    XOM
1 2021-02-01  13192.35  118.57  193.17  154.65  231.84  31.47  60.05  56.07  2.00  141.69  132.26  10.47  35.46  15.89  445.78  225.17  117.37  212.55  43.50  104.02  46.07
2 2021-03-01  13246.87  119.62  213.67  154.70  255.17  35.08  65.54  59.69  6.48  147.87  136.81  11.67  34.17  17.25  452.02  222.64  110.23  214.89  45.75  121.03  48.11
3 2021-04-01  13962.68  128.74  227.71  173.37  263.04  36.93  77.81  56.47  7.60  146.41  138.23  11.28  33.12  17.90  466.01  236.48  109.21  218.20  45.46  130.04  49.32
4 2021-05-01  13748.74  122.03  229.51  161.15  273.24  38.63  82.24  61.55  8.53  152.28  148.47  10.10  33.65  17.15  465.28  208.41  109.79  208.63  44.92  132.59  50.30
5 2021-06-01  14503.95  134.35  242.70  172.01  262.31  37.57  86.21  58.41  7.12  149.14  140.60  10.13  35.52  16.77  499.94  226.57 